## 自定义层

tf官方推荐使用`tf.keras`作为高级抽象API来构建神经网络.这意味着,大多数tf api均可执行在动态图中.

In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 网络层:一些有用的公共操作
许多机器学习模型都可以表示为相对简单的层的组合和堆叠，而TensorFlow既提供了许多公共层的集合，也为您提供了一种简单的方法，可以从头开始编写自己的特定于应用程序的层，也可以是现有层的组合。

在`tf.keras`中包含了全部的keras API,在编译你的模型的时候keras层是非常有用的.


## 实现自定义层

实现自定义层最好的方式就是继承`tf.keras.Layer`类并实现以下方法:

1. `__inti__`这里可以做一些与输入无关的初始化操作

2. `build`这里是你知道输入张量的维度而且你可以初始化的其他部分

3. `call`前向计算

请注意，您不必等到调用`build`来创建变量，您也可以在`__init__`中创建它们。但是，在`build`中创建它们的好处是，它可以根据层将要操作的输入的形状进行后期变量创建。另一方面，在`__init__`中创建变量意味着需要显式指定创建变量所需的形状。

In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]),self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [7]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

如果尽可能使用标准层，则总体代码更易于阅读和维护，因为其他读者将熟悉标准层的行为。

## 模型:多层组合

机器学习模型中许多有趣的层都是通过组合现有层来实现的。例如, resnet的每一个残差块都是通过卷积,皮归一化和连接实现的,层可以嵌套在其他层中。

当你需要`Model.fit` `Model.evaluate` `Model.save`方法时,你可以通过继承`keras.Model`类

另一项功能由`keras.Model`（而不是`keras.layers.Layer`)除了跟踪变量外`keras.Model`还可以跟踪其内部层，使其更易于检查。



In [8]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)

block = ResnetIdentityBlock(1, [1, 2, 3])

In [9]:
_ = block(tf.zeros([1, 3, 3, 3]))

In [10]:
block.layers

In [11]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

然而,大多数情况下,模型可以简单的使用`tf.keras.Sequential`来进行堆栈组合

In [12]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>